# Modeling & Summary
- 목적
> - PUBG Finish Placement Prediction 대회 dataleakage로 인하여, 문제 'killplace'컬럼을 제외한 데이터들에 대해서 모델 성능 개선
> - 기준이 되는 Base model(xgboost 0.090195,0.09180 mae 기준)을 제작하였으며 전처리 및 모델 학습을 통한 성능 개선하였습니다.
- 1.Preprocessing
> - 이상치 제거 DataFrame 불러온다.
> - Groupid 기준으로 target값을 제외한 컬럼에 대해 평균값을 각 gropuid 기준으로 넣어준다.
>> - pubg는 groupid 별로 target값이 생성되기 때문에 group의 전반적인 데이터를 고려할 수 있도록 평균치를 넣어주었습니다.
> - Dataframe을 X,y으로 분리하여 모델학습을 진행한다.
- 2. Model Test
> - model select을 위해 6개의 regression 모델 성능 비교
> - 상위 3개 모델 select
- 3. Model Hyperparameter Tuning & Model ensemble
> - select model Hyperparameter Tuning
> - voting regression을 3개의 모델에 대해서 진행하였습니다.
- 4. Conclusion
> - 최종적으로 Base model(xgboost 0.090195,0.09180 mae 기준)보다 0.024정도 성능 개선하였습니다.

# 1. Preprocessing

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

In [2]:
# 경로설정
df = pd.read_csv('../../data/train_preprocessed.csv')
df = df.dropna()

- 이상치 전처리한 DataFrame을 불러옵니다.

In [3]:
def feature_engineering(df):
    #수치형 데이터
    df_col= [ 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills','matchType',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints']
    # df_t는 데이터를 그룹아이디 기준으로 평균을 내기 위해 저장할 데이터프레임
    df_t=df.copy()
    #라벨인코딩
    le = LabelEncoder()
    df_t['matchType']= le.fit_transform(df_t['matchType'])
    df_t['matchType']
    # 그룹아이디를 기준으로 수치형 컬럼들을 평균화
    df_t= df_t.groupby(['groupId'])[df_col].agg('mean').reset_index()
    
    return df_t


train02 = feature_engineering(df)

train03=df[['groupId','winPlacePerc']]

train03=pd.merge(train03,train02,how='left',on='groupId')
train03=train03.drop(columns=['killPlace','groupId'])

X=train03.drop(columns='winPlacePerc')
y=train03[['winPlacePerc']]

- Groupid 기준으로 target값을 제외한 컬럼에 대해 평균값을 각 gropuid 기준으로 넣어줍니다.
- 이를 통해 group 별로 동일한 컬럼 값을 가지게됩니다.

In [4]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

- Dataframe을 train, valid로 분리합니다.

# 2. Model Test

In [16]:
# 모델에 따른 학습 진행 함수
def cal_mae(X, y, func):
    
    # 학습
    model = func.fit(X, y)
    
    # 모델 평가
    y_pred = model.predict(X_valid)
    return mean_absolute_error(y_valid, y_pred)

In [17]:
# LinearRegression
LR = cal_mae(X, y, LinearRegression())
# Lasso
LS = cal_mae(X, y, Lasso())
# Ridge
RG = cal_mae(X, y, Ridge())
# RandomForest
RDF = cal_mae(X, y, RandomForestRegressor(n_estimators=100,random_state=0, n_jobs= -1))
# XGBRegressor
XGBR = cal_mae(X, y, xgb.XGBRegressor(n_estimators=100, n_jobs= -1))
# LGBMRegressor
LGB = cal_mae(X, y, lgb.LGBMRegressor(n_estimators=100, n_jobs= -1))

print(f'Linear Regressor :  mae = {LR}')
print(f'Lasso :  mae = {LS}')
print(f'Ridge :  mae = {RG}')
print(f'RandomForest :  mae = {RDF}')
print(f'XGBRegressor :  mae = {XGBR}')
print(f'LGBMRegressor :  mae = {LGB}')

/var/folders/tb/j9qmrzv54k73r5l100jxqll80000gn/T/ipykernel_5409/3713937285.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model = func.fit(X, y)


Linear Regressor :  mae = 0.09922127415562758
Lasso :  mae = 0.11740991483931458
Ridge :  mae = 0.09922127466890124
RandomForest :  mae = 0.008669915788687477
XGBRegressor :  mae = 0.06774482224921537
LGBMRegressor :  mae = 0.06908689886200906


- LinearRegression, Lasso, Ridge, RandomForest, XGBRegressor, LGBMRegressor 총 6개 모델에 대해 성능 비교
- RandomForest, XGBRegressor, LGBMRegressor 상위 3개 모델 select 하였습니다.

# 3. Model Hyperparameter Tuning

In [5]:
xgboost = xgb.XGBRegressor(
    reg_lambda = 2.7806808012468585,
    alpha = 0.011386465419313093,
    colsample_bytree = 0.7,
    subsample = 0.8,
    learning_rate = 0.014,
    n_estimators = 750,
    max_depth = 11,
    random_state = 42,
    min_child_weight = 231,
    n_jobs = -1
)

lightgbm = lgb.LGBMRegressor(
    lambda_l1 = 0.00011127147165282962,
    lambda_l2 = 4.381519364899678e-05,
    path_smooth = 1.356576322384448e-07,
    learning_rate = 0.05,
    num_leaves = 138,
    min_data_in_leaf = 55,
    max_bin = 199,
    n_estimators = 751,
    feature_fraction = 0.5160178724035975,
    bagging_fraction = 0.5257183949059583,
    random_state = 42,
    n_jobs= -1,
)

rf = RandomForestRegressor(
    n_estimators = 355,
    random_state = 42,
    max_depth = 10,
    min_samples_leaf = 5,
    min_samples_split = 12
)

- 각 모델별 하이퍼파라미터 튜닝을 진행하였으며 최적의 하이퍼파라미터 기준으로 모델을 생성하였습니다.

In [6]:
votingreg = VotingRegressor([('lightgbm', lightgbm), 
                             ('xgboost', xgboost),
                             ('rf',rf)
                            ],
                           verbose=True,
                            n_jobs=-1
                           )

- 3개 모델에 대해 VotingRegressor 모델 생성

# 4. Conclusion

In [7]:
# votingreg.fit(X_train, y_train.values.ravel())

[LightGBM] [Warning] feature_fraction is set=0.5160178724035975, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5160178724035975
[LightGBM] [Warning] bagging_fraction is set=0.5257183949059583, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5257183949059583
[LightGBM] [Warning] lambda_l2 is set=4.381519364899678e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.381519364899678e-05
[LightGBM] [Warning] min_data_in_leaf is set=55, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=55
[LightGBM] [Warning] lambda_l1 is set=0.00011127147165282962, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00011127147165282962
[Voting] ................. (1 of 3) Processing lightgbm, total= 4.1min
MAE: 0.06697931354289616
[Voting] ....................... (3 of 3) Processing rf, total=98.7min
[Voting] .................. (2 of 3) Processing xgboost, total=98.8min


- voting regressor train 진행하였습니다.

In [9]:
preds = votingreg.predict(X_valid)
print('MAE:', mean_absolute_error(y_valid, preds))

MAE: 0.06697931354289616


- VotingRegressor를 통해 최종 스코어 0.06697931354289616(mae) 입니다.
- 최종적으로 Base model(xgboost 0.090195,0.09180 mae 기준)보다 0.024정도 성능 개선하였습니다.

# 모델 저장

In [35]:
import pickle

# save
# with open('model.pkl','wb') as f:
#     pickle.dump(votingreg,f)

# load
with open('model.pkl', 'rb') as f:
    clf = pickle.load(f)

In [36]:
clf.predict(X_valid)

array([0.00581821, 0.85296118, 0.86305033, ..., 0.8335614 , 0.1363089 ,
       0.50674418])